# Imports

In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline


/Users/Ed/.pyenv/versions/3.10.6/envs/crypto_market_assistant/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Combining the datasets


## Loading the datasets 

Just remember to change the path to the data files

In [44]:
import os
os.path.dirname(os.path.dirname(os.getcwd()))


'/Users/Ed/code/E-Brown96'

In [45]:
os.getcwd()


'/Users/Ed/code/E-Brown96/crypto_market_assistant/notebooks'

In [42]:
route_path = os.path.dirname(os.getcwd())
df_2019 = pd.read_csv(os.path.join(route_path,'Raw_Data','2019_2021_data.csv'))
df_2021 = pd.read_csv('../Raw_Data/2021_2022_data.csv')
df_2022 = pd.read_csv('../Raw_Data/sentimental_data.csv')


In [43]:
df_2019.head()


,cryptocurrency,url,title,text,date,predicted_labels,sentiment,polarity,subjectivity
0,gusd,https://zycrypto.com/mavericks-owner-mark-cuba...,Mavericks Owner Mark Cuban Expects Weekly Tick...,Mark Cuban is confident that his basketball cl...,2021-10-07 16:15:12+00:00,0,Neutral,0.50,0.83
1,dcr,https://beincrypto.com/btc-eth-xrp-sxp-dcr-the...,"BTC, ETH, XRP, SXP, DCR, THETA, SC — Technical...",Bitcoin (BTC) has moved above another major re...,2021-10-06 11:35:26+00:00,0,Neutral,0.04,0.29
2,gusd,https://zycrypto.com/retail-payments-company-v...,Retail Payments Company Verifone Is Rolling Ou...,"Verifone, a global FinTech leader and payment ...",2021-09-29 11:54:45+00:00,0,Neutral,0.00,0.00
3,dcr,https://ambcrypto.com/binance-coin-aave-decred...,"Binance Coin, AAVE, Decred Price Analysis: 23 ...","The likes of Binance Coin, AAVE, and Decred al...",2021-09-23 17:00:06+00:00,0,Neutral,0.00,0.10
4,cspr,https://coinpedia.org/news/will-cardanos-maste...,Will Cardano’s Masterstroke Will Keep It Out O...,Cardano’s platform is ready to take on possibl...,2021-09-23 15:29:06+00:00,0,Neutral,0.00,0.68


# Removing unneeded columns

For 2019 dataset

In [4]:
df_2019.columns


Index(['cryptocurrency', 'url', 'title', 'text', 'date', 'predicted_labels',
       'sentiment', 'polarity', 'subjectivity'],
      dtype='object')

In [5]:
df_2019 = df_2019.drop(columns=['cryptocurrency','url','predicted_labels','sentiment','subjectivity'])


In [6]:
df_2019 = df_2019.rename(columns = {'polarity':'sentiment'})


In [7]:
df_2019.head()


,title,text,date,sentiment
0,Mavericks Owner Mark Cuban Expects Weekly Tick...,Mark Cuban is confident that his basketball cl...,2021-10-07 16:15:12+00:00,0.50
1,"BTC, ETH, XRP, SXP, DCR, THETA, SC — Technical...",Bitcoin (BTC) has moved above another major re...,2021-10-06 11:35:26+00:00,0.04
2,Retail Payments Company Verifone Is Rolling Ou...,"Verifone, a global FinTech leader and payment ...",2021-09-29 11:54:45+00:00,0.00
3,"Binance Coin, AAVE, Decred Price Analysis: 23 ...","The likes of Binance Coin, AAVE, and Decred al...",2021-09-23 17:00:06+00:00,0.00
4,Will Cardano’s Masterstroke Will Keep It Out O...,Cardano’s platform is ready to take on possibl...,2021-09-23 15:29:06+00:00,0.00


For 2021 dataset

In [8]:
df_2021.columns


Index(['date', 'sentiment', 'source', 'subject', 'text', 'title', 'url'], dtype='object')

In [9]:
df_2021 = df_2021.drop(columns=['source','subject','url'])


In [10]:
df_2021.head()


,date,sentiment,text,title
0,2021-10-12 20:00:00,"{'class': 'positive', 'polarity': 0.16, 'subje...","Within a little more than a year, Celo aims to...","Celo to Be Fastest EVM Chain by End of 2022, C..."
1,2021-10-15 00:00:00,"{'class': 'neutral', 'polarity': 0.0, 'subject...",Chinese companies are still topping the blockc...,Tech Crackdown Hasn't Halted Chinese Firms' Bl...
2,2021-10-18 13:58:00,"{'class': 'positive', 'polarity': 0.14, 'subje...",Advancing its project to become \x9caÂ\xa0meta...,"Facebook To Add 10,000 Jobs In EU For Metavers..."
3,2021-10-19 13:39:00,"{'class': 'positive', 'polarity': 0.1, 'subjec...",Banque de France disclosed the results of its ...,French Central Bank's Blockchain Bond Trial Br...
4,2021-10-27 15:17:00,"{'class': 'neutral', 'polarity': 0.0, 'subject...",Cream Finance (CREAM) suffered another flash l...,Cream Finance Suffers Another Exploit as Attac...


In [11]:
df_2022.rename(columns = {'summary':'text','sentiment_score':'sentiment'}, inplace=True)


In [12]:
df_2022


,date,title,text,sentiment
0,20220302T100025,"Millions for Crypto Start-Ups, No Real Names N...",Investors give money to pseudonymous developer...,-0.008880
1,20220302T194621,This Social Club Runs on Crypto Tokens and Vibes,Friends With Benefits is a V.I.P. lounge for c...,0.273382
2,20220303T011835,"Yellen Says Treasury Will Monitor Crypto, Othe...",Yellen suggested the cryptocurrency sector was...,-0.038768
3,20220303T080000,Crypto 'superpower?' India's young investors p...,Crypto 'superpower?' India's young investors p...,0.007835
4,20220303T110013,Buy and Stake These Two Must-Own Cryptos for 8...,Nick Black explains why ALGO and ADA are two o...,0.412672
...,...,...,...,...
5153,20231204T153337,"SAIC Reports Q3 Results, Joins Uber, Wallbox A...","U.S. stocks traded lower, with the Dow Jones i...",0.225342
5154,20231204T153337,"SAIC Reports Q3 Results, Joins Uber, Wallbox A...","U.S. stocks traded lower, with the Dow Jones i...",0.225342
5155,20231204T155000,Stock Market Breaks A Winning Streak; Investor...,Stock Market May Break Win Streak. Investors S...,-0.090532
5156,20231204T155000,Stock Market Breaks A Winning Streak; Investor...,Stock Market May Break Win Streak. Investors S...,-0.090532


All the columns for the 2023 dataset are already in the form we want.

# Converting the date column to just yyyy/mm/dd

In [13]:
df_2019.head()


,title,text,date,sentiment
0,Mavericks Owner Mark Cuban Expects Weekly Tick...,Mark Cuban is confident that his basketball cl...,2021-10-07 16:15:12+00:00,0.50
1,"BTC, ETH, XRP, SXP, DCR, THETA, SC — Technical...",Bitcoin (BTC) has moved above another major re...,2021-10-06 11:35:26+00:00,0.04
2,Retail Payments Company Verifone Is Rolling Ou...,"Verifone, a global FinTech leader and payment ...",2021-09-29 11:54:45+00:00,0.00
3,"Binance Coin, AAVE, Decred Price Analysis: 23 ...","The likes of Binance Coin, AAVE, and Decred al...",2021-09-23 17:00:06+00:00,0.00
4,Will Cardano’s Masterstroke Will Keep It Out O...,Cardano’s platform is ready to take on possibl...,2021-09-23 15:29:06+00:00,0.00


In [14]:
df_2019['date'] = pd.to_datetime(df_2019['date'].str[:-6])
df_2019['date'] = df_2019['date'].dt.strftime('%Y-%m-%d')
df_2019


,title,text,date,sentiment
0,Mavericks Owner Mark Cuban Expects Weekly Tick...,Mark Cuban is confident that his basketball cl...,2021-10-07,0.50
1,"BTC, ETH, XRP, SXP, DCR, THETA, SC — Technical...",Bitcoin (BTC) has moved above another major re...,2021-10-06,0.04
2,Retail Payments Company Verifone Is Rolling Ou...,"Verifone, a global FinTech leader and payment ...",2021-09-29,0.00
3,"Binance Coin, AAVE, Decred Price Analysis: 23 ...","The likes of Binance Coin, AAVE, and Decred al...",2021-09-23,0.00
4,Will Cardano’s Masterstroke Will Keep It Out O...,Cardano’s platform is ready to take on possibl...,2021-09-23,0.00
...,...,...,...,...
404,"Tokenized Gold And Blockchain, Nexo Finance Fi...",Nexo has been known to be the leading DeFi (De...,2020-01-09,0.00
405,Road to Legendary: Top 7 Most Expensive Crypto...,[the_ad id=”36860″] The universe of cryptocurr...,2019-07-10,0.37
406,"Binance Moves 32,888,888 BNB Worth $1.2 Billio...",A massive Binance Coin transfer is showing off...,2019-06-27,0.02
407,Binance About To Issue BTC-Pegged Tokens On It...,Binance has announced its decision to issue to...,2019-06-18,0.00


In [15]:
df_2021['date'] = pd.to_datetime(df_2021['date']).dt.strftime('%Y-%m-%d')
df_2021


,date,sentiment,text,title
0,2021-10-12,"{'class': 'positive', 'polarity': 0.16, 'subje...","Within a little more than a year, Celo aims to...","Celo to Be Fastest EVM Chain by End of 2022, C..."
1,2021-10-15,"{'class': 'neutral', 'polarity': 0.0, 'subject...",Chinese companies are still topping the blockc...,Tech Crackdown Hasn't Halted Chinese Firms' Bl...
2,2021-10-18,"{'class': 'positive', 'polarity': 0.14, 'subje...",Advancing its project to become \x9caÂ\xa0meta...,"Facebook To Add 10,000 Jobs In EU For Metavers..."
3,2021-10-19,"{'class': 'positive', 'polarity': 0.1, 'subjec...",Banque de France disclosed the results of its ...,French Central Bank's Blockchain Bond Trial Br...
4,2021-10-27,"{'class': 'neutral', 'polarity': 0.0, 'subject...",Cream Finance (CREAM) suffered another flash l...,Cream Finance Suffers Another Exploit as Attac...
...,...,...,...,...
3518,2022-03-31,"{'class': 'neutral', 'polarity': 0.0, 'subject...","Ted Cruz submitted a companion legislation, wh...",Sen. Ted Cruz Introduces a Bill to Prevent the...
3519,2022-03-31,"{'class': 'negative', 'polarity': -0.18, 'subj...",The firm previously said its reserves would be...,Circle selects BNY Mellon as custodian for USD...
3520,2022-03-31,"{'class': 'neutral', 'polarity': 0.0, 'subject...","The world’s largest digital asset manager, Gra...",Grayscale Comapres Ethereum to New York City
3521,2022-03-31,"{'class': 'neutral', 'polarity': 0.0, 'subject...","While supporting crypto mining companies, the ...",Bitcoin mining finds fresh support from Russia...


In [16]:
df_2022['date'] = df_2022['date'].apply(lambda x: datetime.strptime(x, "%Y%m%dT%H%M%S").strftime('%Y-%m-%d'))
df_2022


,date,title,text,sentiment
0,2022-03-02,"Millions for Crypto Start-Ups, No Real Names N...",Investors give money to pseudonymous developer...,-0.008880
1,2022-03-02,This Social Club Runs on Crypto Tokens and Vibes,Friends With Benefits is a V.I.P. lounge for c...,0.273382
2,2022-03-03,"Yellen Says Treasury Will Monitor Crypto, Othe...",Yellen suggested the cryptocurrency sector was...,-0.038768
3,2022-03-03,Crypto 'superpower?' India's young investors p...,Crypto 'superpower?' India's young investors p...,0.007835
4,2022-03-03,Buy and Stake These Two Must-Own Cryptos for 8...,Nick Black explains why ALGO and ADA are two o...,0.412672
...,...,...,...,...
5153,2023-12-04,"SAIC Reports Q3 Results, Joins Uber, Wallbox A...","U.S. stocks traded lower, with the Dow Jones i...",0.225342
5154,2023-12-04,"SAIC Reports Q3 Results, Joins Uber, Wallbox A...","U.S. stocks traded lower, with the Dow Jones i...",0.225342
5155,2023-12-04,Stock Market Breaks A Winning Streak; Investor...,Stock Market May Break Win Streak. Investors S...,-0.090532
5156,2023-12-04,Stock Market Breaks A Winning Streak; Investor...,Stock Market May Break Win Streak. Investors S...,-0.090532


Now all the dates have been converted to year month day so we can combine the datasets together.

# Rearranging columns

This step is just necessary so we can concatenate the dataframes together.

In [17]:
column_order = ['date','title','text','sentiment']


In [18]:
df_2019 = df_2019[column_order]
df_2021 = df_2021[column_order]
df_2022 = df_2022[column_order]


# Concatenating the Dataframes

All the dataframes have the same columns and are arranged so they can just be added together.

In [19]:
combined_df = pd.concat([df_2019,df_2021,df_2022])


In [20]:
combined_df


,date,title,text,sentiment
0,2021-10-07,Mavericks Owner Mark Cuban Expects Weekly Tick...,Mark Cuban is confident that his basketball cl...,0.5
1,2021-10-06,"BTC, ETH, XRP, SXP, DCR, THETA, SC — Technical...",Bitcoin (BTC) has moved above another major re...,0.04
2,2021-09-29,Retail Payments Company Verifone Is Rolling Ou...,"Verifone, a global FinTech leader and payment ...",0.0
3,2021-09-23,"Binance Coin, AAVE, Decred Price Analysis: 23 ...","The likes of Binance Coin, AAVE, and Decred al...",0.0
4,2021-09-23,Will Cardano’s Masterstroke Will Keep It Out O...,Cardano’s platform is ready to take on possibl...,0.0
...,...,...,...,...
5153,2023-12-04,"SAIC Reports Q3 Results, Joins Uber, Wallbox A...","U.S. stocks traded lower, with the Dow Jones i...",0.225342
5154,2023-12-04,"SAIC Reports Q3 Results, Joins Uber, Wallbox A...","U.S. stocks traded lower, with the Dow Jones i...",0.225342
5155,2023-12-04,Stock Market Breaks A Winning Streak; Investor...,Stock Market May Break Win Streak. Investors S...,-0.090532
5156,2023-12-04,Stock Market Breaks A Winning Streak; Investor...,Stock Market May Break Win Streak. Investors S...,-0.090532


And now the data can be sorted by date.

In [21]:
sentimental_df = combined_df.sort_values('date').reset_index()
sentimental_df


,index,date,title,text,sentiment
0,408,2019-01-08,Bitcoin's True Value Getting Lost in the Shuff...,Bitcoin's potential to become a store of value...,0.12
1,407,2019-06-18,Binance About To Issue BTC-Pegged Tokens On It...,Binance has announced its decision to issue to...,0.0
2,406,2019-06-27,"Binance Moves 32,888,888 BNB Worth $1.2 Billio...",A massive Binance Coin transfer is showing off...,0.02
3,405,2019-07-10,Road to Legendary: Top 7 Most Expensive Crypto...,[the_ad id=”36860″] The universe of cryptocurr...,0.37
4,404,2020-01-09,"Tokenized Gold And Blockchain, Nexo Finance Fi...",Nexo has been known to be the leading DeFi (De...,0.0
...,...,...,...,...,...
9085,5131,2023-12-04,Crypto stocks surge as bitcoin hits fresh 2023...,Representations of cryptocurrencies are seen i...,0.22334
9086,5132,2023-12-04,Crypto stocks surge as bitcoin hits fresh 2023...,Representations of cryptocurrencies are seen i...,0.22334
9087,5133,2023-12-04,Crypto stocks surge as bitcoin hits fresh 2023...,Representations of cryptocurrencies are seen i...,0.22334
9088,5125,2023-12-04,Futures dip as focus shifts to economic data f...,The New York Stock Exchange ( NYSE ) in New Yo...,0.130546


# Sentimental Analysis 

To analyse the text we are going to use a hugging face model. For the inital model 'ElKulako/cryptobert' was chosen as it was trained on social media related crypto posts. It results in a score and a classification of Bearish, Bullish or Neutral.

In [22]:
tokenizer = BertTokenizer.from_pretrained("kk08/CryptoBERT")
model = BertForSequenceClassification.from_pretrained("kk08/CryptoBERT")

classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)


In [23]:
sentimental_df['title_prediction'] = sentimental_df['title'].apply(lambda x: classifier(x))
sentimental_df['text_prediction'] = sentimental_df['text'].apply(lambda x: classifier(x))
sentimental_df


,index,date,title,text,sentiment,title_prediction,text_prediction
0,408,2019-01-08,Bitcoin's True Value Getting Lost in the Shuff...,Bitcoin's potential to become a store of value...,0.12,"[{'label': 'LABEL_0', 'score': 0.9766811728477...","[{'label': 'LABEL_1', 'score': 0.6001262664794..."
1,407,2019-06-18,Binance About To Issue BTC-Pegged Tokens On It...,Binance has announced its decision to issue to...,0.0,"[{'label': 'LABEL_1', 'score': 0.9333034753799...","[{'label': 'LABEL_1', 'score': 0.9410629868507..."
2,406,2019-06-27,"Binance Moves 32,888,888 BNB Worth $1.2 Billio...",A massive Binance Coin transfer is showing off...,0.02,"[{'label': 'LABEL_1', 'score': 0.7670107483863...","[{'label': 'LABEL_1', 'score': 0.8967406749725..."
3,405,2019-07-10,Road to Legendary: Top 7 Most Expensive Crypto...,[the_ad id=”36860″] The universe of cryptocurr...,0.37,"[{'label': 'LABEL_1', 'score': 0.6037188172340...","[{'label': 'LABEL_1', 'score': 0.9608872532844..."
4,404,2020-01-09,"Tokenized Gold And Blockchain, Nexo Finance Fi...",Nexo has been known to be the leading DeFi (De...,0.0,"[{'label': 'LABEL_1', 'score': 0.9513247013092...","[{'label': 'LABEL_1', 'score': 0.9656965136528..."
...,...,...,...,...,...,...,...
9085,5131,2023-12-04,Crypto stocks surge as bitcoin hits fresh 2023...,Representations of cryptocurrencies are seen i...,0.22334,"[{'label': 'LABEL_1', 'score': 0.9578972458839...","[{'label': 'LABEL_1', 'score': 0.7355278134346..."
9086,5132,2023-12-04,Crypto stocks surge as bitcoin hits fresh 2023...,Representations of cryptocurrencies are seen i...,0.22334,"[{'label': 'LABEL_1', 'score': 0.9578972458839...","[{'label': 'LABEL_1', 'score': 0.7355278134346..."
9087,5133,2023-12-04,Crypto stocks surge as bitcoin hits fresh 2023...,Representations of cryptocurrencies are seen i...,0.22334,"[{'label': 'LABEL_1', 'score': 0.9578972458839...","[{'label': 'LABEL_1', 'score': 0.7355278134346..."
9088,5125,2023-12-04,Futures dip as focus shifts to economic data f...,The New York Stock Exchange ( NYSE ) in New Yo...,0.130546,"[{'label': 'LABEL_0', 'score': 0.9536192417144...","[{'label': 'LABEL_1', 'score': 0.9042448401451..."


In [24]:
def map_to_score(row):
    label = row[0]['label']
    score = row[0]['score']

    if label == 'LABEL_1':
        return score
    else:
        return -score


In [25]:
scored_df = sentimental_df.copy()


In [26]:
scored_df['title_prediction'][0]


[{'label': 'LABEL_0', 'score': 0.9766811728477478}]

In [27]:
scored_df['title_prediction'][0][0]['label']


'LABEL_0'

In [28]:
scored_df['title_prediction'][0][0]['score']


0.9766811728477478

In [29]:
scored_df['scored_title'] = scored_df['title_prediction'].apply(map_to_score)
scored_df['scored_text'] = scored_df['text_prediction'].apply(map_to_score)
scored_df


,index,date,title,text,sentiment,title_prediction,text_prediction,scored_title,scored_text
0,408,2019-01-08,Bitcoin's True Value Getting Lost in the Shuff...,Bitcoin's potential to become a store of value...,0.12,"[{'label': 'LABEL_0', 'score': 0.9766811728477...","[{'label': 'LABEL_1', 'score': 0.6001262664794...",-0.976681,0.600126
1,407,2019-06-18,Binance About To Issue BTC-Pegged Tokens On It...,Binance has announced its decision to issue to...,0.0,"[{'label': 'LABEL_1', 'score': 0.9333034753799...","[{'label': 'LABEL_1', 'score': 0.9410629868507...",0.933303,0.941063
2,406,2019-06-27,"Binance Moves 32,888,888 BNB Worth $1.2 Billio...",A massive Binance Coin transfer is showing off...,0.02,"[{'label': 'LABEL_1', 'score': 0.7670107483863...","[{'label': 'LABEL_1', 'score': 0.8967406749725...",0.767011,0.896741
3,405,2019-07-10,Road to Legendary: Top 7 Most Expensive Crypto...,[the_ad id=”36860″] The universe of cryptocurr...,0.37,"[{'label': 'LABEL_1', 'score': 0.6037188172340...","[{'label': 'LABEL_1', 'score': 0.9608872532844...",0.603719,0.960887
4,404,2020-01-09,"Tokenized Gold And Blockchain, Nexo Finance Fi...",Nexo has been known to be the leading DeFi (De...,0.0,"[{'label': 'LABEL_1', 'score': 0.9513247013092...","[{'label': 'LABEL_1', 'score': 0.9656965136528...",0.951325,0.965697
...,...,...,...,...,...,...,...,...,...
9085,5131,2023-12-04,Crypto stocks surge as bitcoin hits fresh 2023...,Representations of cryptocurrencies are seen i...,0.22334,"[{'label': 'LABEL_1', 'score': 0.9578972458839...","[{'label': 'LABEL_1', 'score': 0.7355278134346...",0.957897,0.735528
9086,5132,2023-12-04,Crypto stocks surge as bitcoin hits fresh 2023...,Representations of cryptocurrencies are seen i...,0.22334,"[{'label': 'LABEL_1', 'score': 0.9578972458839...","[{'label': 'LABEL_1', 'score': 0.7355278134346...",0.957897,0.735528
9087,5133,2023-12-04,Crypto stocks surge as bitcoin hits fresh 2023...,Representations of cryptocurrencies are seen i...,0.22334,"[{'label': 'LABEL_1', 'score': 0.9578972458839...","[{'label': 'LABEL_1', 'score': 0.7355278134346...",0.957897,0.735528
9088,5125,2023-12-04,Futures dip as focus shifts to economic data f...,The New York Stock Exchange ( NYSE ) in New Yo...,0.130546,"[{'label': 'LABEL_0', 'score': 0.9536192417144...","[{'label': 'LABEL_1', 'score': 0.9042448401451...",-0.953619,0.904245


In [30]:
scored_df['average_score'] = (scored_df['scored_title'] + scored_df['scored_text'])/2
scored_df


,index,date,title,text,sentiment,title_prediction,text_prediction,scored_title,scored_text,average_score
0,408,2019-01-08,Bitcoin's True Value Getting Lost in the Shuff...,Bitcoin's potential to become a store of value...,0.12,"[{'label': 'LABEL_0', 'score': 0.9766811728477...","[{'label': 'LABEL_1', 'score': 0.6001262664794...",-0.976681,0.600126,-0.188277
1,407,2019-06-18,Binance About To Issue BTC-Pegged Tokens On It...,Binance has announced its decision to issue to...,0.0,"[{'label': 'LABEL_1', 'score': 0.9333034753799...","[{'label': 'LABEL_1', 'score': 0.9410629868507...",0.933303,0.941063,0.937183
2,406,2019-06-27,"Binance Moves 32,888,888 BNB Worth $1.2 Billio...",A massive Binance Coin transfer is showing off...,0.02,"[{'label': 'LABEL_1', 'score': 0.7670107483863...","[{'label': 'LABEL_1', 'score': 0.8967406749725...",0.767011,0.896741,0.831876
3,405,2019-07-10,Road to Legendary: Top 7 Most Expensive Crypto...,[the_ad id=”36860″] The universe of cryptocurr...,0.37,"[{'label': 'LABEL_1', 'score': 0.6037188172340...","[{'label': 'LABEL_1', 'score': 0.9608872532844...",0.603719,0.960887,0.782303
4,404,2020-01-09,"Tokenized Gold And Blockchain, Nexo Finance Fi...",Nexo has been known to be the leading DeFi (De...,0.0,"[{'label': 'LABEL_1', 'score': 0.9513247013092...","[{'label': 'LABEL_1', 'score': 0.9656965136528...",0.951325,0.965697,0.958511
...,...,...,...,...,...,...,...,...,...,...
9085,5131,2023-12-04,Crypto stocks surge as bitcoin hits fresh 2023...,Representations of cryptocurrencies are seen i...,0.22334,"[{'label': 'LABEL_1', 'score': 0.9578972458839...","[{'label': 'LABEL_1', 'score': 0.7355278134346...",0.957897,0.735528,0.846713
9086,5132,2023-12-04,Crypto stocks surge as bitcoin hits fresh 2023...,Representations of cryptocurrencies are seen i...,0.22334,"[{'label': 'LABEL_1', 'score': 0.9578972458839...","[{'label': 'LABEL_1', 'score': 0.7355278134346...",0.957897,0.735528,0.846713
9087,5133,2023-12-04,Crypto stocks surge as bitcoin hits fresh 2023...,Representations of cryptocurrencies are seen i...,0.22334,"[{'label': 'LABEL_1', 'score': 0.9578972458839...","[{'label': 'LABEL_1', 'score': 0.7355278134346...",0.957897,0.735528,0.846713
9088,5125,2023-12-04,Futures dip as focus shifts to economic data f...,The New York Stock Exchange ( NYSE ) in New Yo...,0.130546,"[{'label': 'LABEL_0', 'score': 0.9536192417144...","[{'label': 'LABEL_1', 'score': 0.9042448401451...",-0.953619,0.904245,-0.024687


In [33]:
processed_text_df = scored_df[['date','scored_title','scored_text','average_score']]
processed_text_df


,date,scored_title,scored_text,average_score
0,2019-01-08,-0.976681,0.600126,-0.188277
1,2019-06-18,0.933303,0.941063,0.937183
2,2019-06-27,0.767011,0.896741,0.831876
3,2019-07-10,0.603719,0.960887,0.782303
4,2020-01-09,0.951325,0.965697,0.958511
...,...,...,...,...
9085,2023-12-04,0.957897,0.735528,0.846713
9086,2023-12-04,0.957897,0.735528,0.846713
9087,2023-12-04,0.957897,0.735528,0.846713
9088,2023-12-04,-0.953619,0.904245,-0.024687


In [35]:
final_text_df = processed_text_df.groupby('date', as_index=False).mean()
final_text_df


,date,scored_title,scored_text,average_score
0,2019-01-08,-0.976681,0.600126,-0.188277
1,2019-06-18,0.933303,0.941063,0.937183
2,2019-06-27,0.767011,0.896741,0.831876
3,2019-07-10,0.603719,0.960887,0.782303
4,2020-01-09,0.951325,0.965697,0.958511
...,...,...,...,...
985,2023-11-30,0.631888,0.589097,0.610492
986,2023-12-01,0.741688,0.752807,0.747247
987,2023-12-02,-0.002369,-0.004065,-0.003217
988,2023-12-03,0.324950,0.944111,0.634530
